In [1]:
import os
from data_partitioner_jsonl import *
from lisa_tagging_reddit import *

In [2]:
file_name = 'corpus.jsonl'
dataset = "long-reddit"

In [3]:
%time
# input_path = os.path.join(f'/shared/3/projects/hiatus/idiolect/data/full_pilot/{dataset}', file_name)
# partition_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/{dataset}', file_name.split('.')[0]) 
# tag_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-{dataset}', file_name.split('.')[0])
# output_dir= f'/shared/3/projects/hiatus/tagged_data/lisa-{dataset}'
input_path = os.path.join(f'/shared/3/projects/hiatus/idiolect/data/full_pilot/{dataset}', file_name)
partition_dir = os.path.join(os.path.join(f'/shared/3/projects/hiatus/tagged_data/partitions/{dataset}', file_name.split('.')[0]), 'merged') 
tag_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-{dataset}', file_name.split('.')[0])
output_dir= f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-{dataset}'
# ensures all directories will exist
for directory in [partition_dir, tag_dir, output_dir]:
    print(directory)
    os.makedirs(directory, exist_ok=True)


CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 5.01 µs
/shared/3/projects/hiatus/tagged_data/partitions/long-reddit/corpus/merged
/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-long-reddit/corpus
/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-long-reddit


## Partition the file into 50 parts

In [4]:
# %%time
# partition_file(input_path, partition_dir, chunks=50)

In [5]:
# from tqdm import tqdm  # Import tqdm for progress bar
# import time
# import json
# import os
# import torch

# from tokenization_enc_t5 import EncT5Tokenizer
# from modeling_enc_t5 import EncT5ForSequenceClassification

# tokenizer = EncT5Tokenizer.from_pretrained("t5-base")
# model = EncT5ForSequenceClassification.from_pretrained("/shared/3/projects/hiatus/idiolect/models/stylegenome_lisa_sfam/lisa_checkpoint", num_labels=768, problem_type="regression")
# model.eval()

# os.nice(0)

# def tag_lisa(obj):
#     start_time = time.time()  # Start timing
#     # This will return a 768-dimensional LISA vector for the text, where each element is a score from 0-1, where the score corresponds to each concept in "lisa_dimensions.json".
#     with torch.no_grad():
#         tokenized = tokenizer(
#             [obj['body']],
#             truncation=True, max_length=512, padding=True, return_tensors="pt"
#         )
#         prediction = model.forward(**tokenized)[0][0].cpu().detach().float()
#         vector = torch.clamp(prediction, min=0.0, max=1.0).tolist()
#     obj['lisa_vector'] = vector
#     return obj

# in_file_name = 'partition-21-30.jsonl'

# input_file = f'{partition_dir}/{in_file_name}'
# output_file = f'{output_dir}/{in_file_name}'

# chunk_size = 50000
# chunk = []
# skip_lines = 100000  # Variable to skip n lines before beginning to tag

# def append_chunk(output_file, chunk):
#     try:
#         with open(output_file, 'a', encoding='utf-8') as writer:
#             for obj in chunk:
#                 writer.write(json.dumps(obj) + '\n')
#         print(f"Appended {len(chunk)} records to {output_file}.")
#     except Exception as e:
#         print(f"Error appending chunk to {output_file}: {e}")

# with open(input_file, 'r', encoding='utf-8') as infile:
#     for _ in range(skip_lines):  # Skip the first n lines
#         next(infile)

#     for line in tqdm(infile, desc="Processing lines"):
#         json_obj = json.loads(line.strip())  # Load JSON object from current line
#         if 'bot' in json_obj['author'].lower():
#             continue
#         tagged_obj = tag_lisa(json_obj) 
#         chunk.append(tagged_obj)
        
#         if len(chunk) >= chunk_size:
#             append_chunk(output_file, chunk)
#             chunk = []

#     if chunk:
#         append_chunk(output_file, chunk)


In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar
import time
import json
import os
import torch

from tokenization_enc_t5 import EncT5Tokenizer
from modeling_enc_t5 import EncT5ForSequenceClassification

tokenizer = EncT5Tokenizer.from_pretrained("t5-base")
model = EncT5ForSequenceClassification.from_pretrained("/shared/3/projects/hiatus/idiolect/models/stylegenome_lisa_sfam/lisa_checkpoint", num_labels=768, problem_type="regression")
model.eval()

os.nice(0)

def tag_lisa(obj):
    start_time = time.time()  # Start timing
    # This will return a 768-dimensional LISA vector for the text, where each element is a score from 0-1, where the score corresponds to each concept in "lisa_dimensions.json".
    with torch.no_grad():
        tokenized = tokenizer(
            [obj['body']],
            truncation=True, max_length=512, padding=True, return_tensors="pt"
        )
        prediction = model.forward(**tokenized)[0][0].cpu().detach().float()
        vector = torch.clamp(prediction, min=0.0, max=1.0).tolist()
    obj['lisa_vector'] = vector
    return obj

in_file_name = 'partition-31-40.jsonl'

input_file = f'{partition_dir}/{in_file_name}'
output_file = f'{output_dir}/{in_file_name}'

chunk_size = 50000
chunk = []
skip_lines = 400000  # Variable to skip n lines before beginning to tag

def append_chunk(output_file, chunk):
    try:
        with open(output_file, 'a', encoding='utf-8') as writer:
            for obj in chunk:
                writer.write(json.dumps(obj) + '\n')
        print(f"Appended {len(chunk)} records to {output_file}.")
    except Exception as e:
        print(f"Error appending chunk to {output_file}: {e}")

with open(input_file, 'r', encoding='utf-8') as infile:
    for _ in range(skip_lines):  # Skip the first n lines
        next(infile)

    for line in tqdm(infile, desc="Processing lines"):
        json_obj = json.loads(line.strip())  # Load JSON object from current line
        if 'bot' in json_obj['author'].lower():
            continue
        tagged_obj = tag_lisa(json_obj) 
        chunk.append(tagged_obj)
        
        if len(chunk) >= chunk_size:
            append_chunk(output_file, chunk)
            chunk = []

    if chunk:
        append_chunk(output_file, chunk)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'EncT5Tokenizer'.
You are using the default legacy behaviour of the <class 'tokenization_enc_t5.EncT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing lines: 472it [00:55,  9.14it/s]

In [ ]:
# from tqdm import tqdm  # Import tqdm for progress bar
# import time
# import json
# import os
# import torch

# from tokenization_enc_t5 import EncT5Tokenizer
# from modeling_enc_t5 import EncT5ForSequenceClassification

# tokenizer = EncT5Tokenizer.from_pretrained("t5-base")
# model = EncT5ForSequenceClassification.from_pretrained("/shared/3/projects/hiatus/idiolect/models/stylegenome_lisa_sfam/lisa_checkpoint", num_labels=768, problem_type="regression")
# model.eval()

# os.nice(0)

# def tag_lisa(obj):
# #     start_time = time.time()  # Start timing
#     # This will return a 768-dimensional LISA vector for the text, where each element is a score from 0-1, where the score corresponds to each concept in "lisa_dimensions.json".
#     with torch.no_grad():
#         tokenized = tokenizer(
#             [obj['body']],
#             truncation=True, max_length=512, padding=True, return_tensors="pt"
#         )
#         prediction = model.forward(**tokenized)[0][0].cpu().detach().float()
#         vector = torch.clamp(prediction, min=0.0, max=1.0).tolist()
#     end_time = time.time()  # End timing
#     elapsed_time = end_time - start_time  # Calculate elapsed time
#     print(f"Time taken to tag document {obj['id']}: {elapsed_time:.4f} seconds")
#     obj['lisa_vector'] = vector
#     return obj

# # datasets = ["long-reddit"]
# in_file_name = 'partition-1-10.jsonl'

# input_file = f'{partition_dir}/{in_file_name}'
# output_file = f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/{in_file_name.split(".")[0]}.jsonl'

# buffer_size = 100  # Number of JSON objects to buffer before writing to file
# buffer = []

# with open(input_file, 'r', encoding='utf-8') as infile, \
# open(output_file, 'w', encoding='utf-8') as outfile:
#     for line in tqdm(infile, desc="Processing lines"):
#         json_obj = json.loads(line.strip())  # Load JSON object from current line
#         mutated_obj = tag_lisa(json_obj)  # Mutate JSON object
#         buffer.append(mutated_obj)
        
#         if len(buffer) >= buffer_size:
#             outfile.write('\n'.join(json.dumps(obj, ensure_ascii=False) for obj in buffer) + '\n')
#             buffer = []

#     # Write any remaining objects in the buffer
#     if buffer:
#         outfile.write('\n'.join(json.dumps(obj, ensure_ascii=False) for obj in buffer) + '\n')


## Tag each partition with 1 CPU 

Set nice value low so we don't hog the server


In [ ]:
# %%time
# import logging
# original_level = logging.getLogger().getEffectiveLevel()
# logging.getLogger().setLevel(logging.ERROR)

# tag_partitions(
#                input_directory=partition_dir,
#                output_directory=tag_dir,
#                num_workers=5,
# #                default_niceness=0
#                )
# logging.getLogger().setLevel(original_level)


## Join the partitioned files

In [ ]:
# join_tagged_files(input_directory=tag_dir,
#                   output_file=os.path.join(output_dir, file_name))

## Delete the temp directories

In [ ]:
# delete_partitioned_files(partition_dir)
# delete_partitioned_files(tag_dir)